In [2]:
import numpy as np
import copy
import pickle
import time
from sklearn.cross_validation import train_test_split
from sklearn.metrics import roc_curve,roc_auc_score
import random
import logging
from function import *

class LSPLM:

    def __init__(self,
                 pieceNum = 12,
                 iterNum = 1000,
                 intercept = True,
                 beta1 = 0.1,
                 beta2 = 0.1,
                 alpha = 0.001,
                 epison = 10e-8,
                 lamb = 0.1,
                 beta = 0.1,
                 terminate = False
                 ):
        """
        :param feaNum:  特征数
        :param classNum:    类别数
        :param iterNum:
        :param intercept:
        :param memoryNum:
        :param beta:
        :param lamb:
        :param u_stdev:
        :param w_stdev:
        """
        self.pieceNum = pieceNum
        self.iterNum = iterNum
        self.intercept = intercept
        self.beta = beta
        self.lamb = lamb
        self.beta1 = beta1
        self.beta2 = beta2
        self.alpha = alpha
        self.N = 0
        self.p = 0
        self.terminate = terminate
        self.epison = epison

    def fit(self,X, y):
        """
            训练ls-plm large scale piece-wise linear model
        :param data:
        :return:
        """

        # np.random.seed(0)
        N, p = X.shape
        self.N = N
        if self.intercept:
            self.p = p + 1
            pad = np.ones((N, p + 1))
            pad[:,:-1] = X
            X = pad
            del pad
        else:
            self.p = p

        ## Intialization
        np.random.seed(0)
        weight_W = np.random.normal(0,0.1, (self.pieceNum, self.p))
        weight_U = np.random.normal(0,0.1, (self.pieceNum, self.p))
        best_weight_W = np.random.normal(0,0.1, (self.pieceNum, self.p))
        best_weight_U = np.random.normal(0,0.1, (self.pieceNum, self.p))
        m_w = np.zeros((self.pieceNum, self.p))
        m_u = np.zeros((self.pieceNum, self.p))
        v_w = np.zeros((self.pieceNum, self.p))
        v_u = np.zeros((self.pieceNum, self.p))
        loss_before = calLoss(X, y, weight_W, weight_U, self.lamb, self.beta)
        GW, GU = sumCalDerivative(weight_W, weight_U, X, y)
        LW, LU = virtualGradient(weight_W, weight_U, GW, GU,self.beta,self.lamb)
        it = 1
        del GW,GU
        loss_best = np.maximum
        best_iter = 0
        optimize_counter = 0
        while it < self.iterNum:
            print "iter:%d, loss:%s" % (it, loss_before)
            start_time = time.time()
            
            GW, GU = sumCalDerivative(weight_W, weight_U, X, y)
            newLW, newLU = virtualGradient(weight_W, weight_U, GW, GU,self.beta,self.lamb)
            del GW,GU


            PW, PU = adam(newLW, newLU, m_w, m_u, v_w, v_u, self.beta1, self.beta2, it, self.alpha, self.epison)

            new_weight_W, new_weight_U = weight_W + PW, weight_U + PU

            del PW, PU

            new_weight_W = fixOrthant(newLW, weight_W, new_weight_W)
            new_weight_U = fixOrthant(newLU, weight_U, new_weight_U)
            loss_now = calLoss(X, y, new_weight_W, new_weight_U, self.lamb, self.beta)
            if(loss_now < loss_best):
                loss_best = loss_now
                best_iter = it
                best_weight_W = new_weight_W
                best_weight_U = new_weight_U
            if(loss_before < loss_now):
                optimize_counter += 1
                if(optimize_counter >= 10):
                    self.weight_U = best_weight_U
                    self.weight_W = best_weight_W
                    print("use time: ", time.time() - start_time)
                    print("The best result get at %d iteration with loss: %f" % (best_iter, loss_best))
                    return "Done!"
            else:
                optimize_counter = 0
            

            weight_U = new_weight_U
            weight_W = new_weight_W
            LW = newLW
            LU = newLU
            del newLW
            del newLU
            del new_weight_U
            del new_weight_W
            loss_before = loss_now
            

            it += 1


        logging.info("============iterator : %s end ==========" % it)
        print("")

        print("The best result get at %d iteration with loss: %f" % (best_iter, loss_best))
        print("Done!")
        self.weight_W = best_weight_W
        self.weight_U = best_weight_U
        
    def predict_proba(self, X):
        N, p = X.shape
        if self.intercept:
            pad = np.ones((N, p + 1))
            pad[:,:-1] = X
            X = pad
            del pad
        return mlr_total(self.weight_W,self.weight_U,X)

    def predict(self, X):
        return np.array(self.predict_proba(X) > 0.5, dtype = int)

/home/chris/anaconda2/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [3]:
# test
X1 = np.random.normal(1,0.2, (10000, 1000))
X2 = np.random.normal(-1,0.2, (10000,1000))
X = np.vstack((X1,X2))
y = np.zeros(20000)
for i in range(0,10000):
    y[i] = 1
ls = LSPLM(iterNum=1000,lamb = 1,beta = 1,pieceNum=2)
ls.fit(X,y)
y_test = ls.predict_proba(X)

(31763.520707691379, 6.338252818179253, 226.84178852281386)
iter:1, loss:31996.700749
(17032.023684845692, 6.3356291373790183, 226.72795536359831)
iter:2, loss:17265.0872693
(14859.644329980589, 6.3337337224285806, 226.63342593881879)
iter:3, loss:15092.6114896
(10982.505514385306, 6.3335292448600065, 226.57538654364137)
iter:4, loss:11215.4144302
(7673.5931988335278, 6.3324171111012779, 226.50214134863495)
iter:5, loss:7906.42775729
(4552.7988933516144, 6.3324085044951213, 226.45583344734831)
iter:6, loss:4785.5871353
(2185.3385715323825, 6.3316132258462634, 226.38357848964768)
iter:7, loss:2418.05376325
(938.07544823471767, 6.3312009207074817, 226.3165970482151)
iter:8, loss:1170.7232462
(387.92248717573875, 6.3299524117381445, 226.2121217858878)
iter:9, loss:620.464561373
(160.78054924516235, 6.3254750088627798, 225.96979153905906)
iter:10, loss:393.075815793
(68.058280020086116, 6.3112861037965731, 225.32365077615316)
iter:11, loss:299.6932169
(29.544337758510505, 6.281058191722066